In [1]:
import numpy as np
import cv2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import salientregions as sr
import scipy.io as sio

In [2]:
#Load the image
path_to_image = '../../../TestData/Binary/Binary_holes.png'
holesim = sr.binarize(cv2.imread(path_to_image), threshold=128, visualize=False)
path_to_image_islands = '../../../TestData/Binary/Binary_islands.png'
islandsim = sr.binarize(cv2.imread(path_to_image_islands), threshold=128, visualize=False)
path_to_image_example = '../../../TestData/Binary/Binary_all_types_noise.png'
allim = sr.binarize(cv2.imread(path_to_image_example), threshold=128, visualize=False)
path_to_image_indpred = '../../../TestData/Binary/Binary_indentations_protrusions.png'
indpredim = sr.binarize(cv2.imread(path_to_image_indpred), threshold=128, visualize=False)
path_to_image_nested = '../../../TestData/Binary/Binary_nested.png'
nestedim = sr.binarize(cv2.imread(path_to_image_nested), threshold=128, visualize=False)

error: /home/dafne/OpenCV/opencv-3.1.0/modules/imgproc/src/thresh.cpp:1204: error: (-215) src.type() == CV_8UC1 in function threshold


In [4]:
img = nestedim
sr.show_image(img)

In [5]:
#Define parameters and get structuring elements
connectivity = 4
area_factor = 0.05
SE_size_factor = 0.075
SE, lam = sr.get_SE(img, SE_size_factor)
SEhi, lamhi = sr.get_SEhi(SE, lam)
print 'lambda is: %f' % lam
print 'SE size is: %f' % SE.shape[0]
print 'lambdahi is: %f' % lamhi
print 'SEhi size is: %f' % SEhi.shape[0]

lambda is: 80.000000
SE size is: 31.000000
lambdahi is: 8.000000
SEhi size is: 15.000000


In [6]:
SE_mat = sio.loadmat("../../../TestData/Binary/SE_neighb_nested.mat")['SE_n'] 
SEhi_mat = sio.loadmat("../../../TestData/Binary/SEhi_neighb_nested.mat")['SEhi_n']
print 'SE size is: %i' % SE_mat.shape[0]
print 'SEhi size is: %i' % SEhi_mat.shape[0]

SE size is: 31
SEhi size is: 15


In [7]:
#get holes
filled, holes = sr.get_holes(img, lam=lam, visualize=False)
#sr.show_image(holes)

In [8]:
#get islands
filledinv, islands = sr.get_islands(img, lam=lam, visualize=False)
#sr.show_image(islands)

In [9]:
#filled, prots = sr.get_protrusions(img, filled=filled, lam=lam, visualize=True)
filled, prots = sr.get_protrusions(img, filled, holes, SE=SE_mat, SEhi=SEhi_mat, lam=lam, visualize=False)
filledinv, indents = sr.get_indentations(img, filledinv, islands, SE=SE_mat, SEhi=SEhi_mat, lam=lam, visualize=True)

In [10]:
holes_true, islands_true, indents_true, prots_true = \
    sr.read_matfile('../../../TestData/Binary/Binary_nested_binregions.mat', visualize=False)

In [11]:
elements_visualized = sr.visualize_elements(img, holes=holes, islands=islands, protrusions=prots, 
                                            indentations=indents, display_name='detected salient regions')
trueelements_visualized = sr.visualize_elements(img, 
                                             holes=holes_true, islands=islands_true, 
                                             protrusions=prots_true, indentations=indents_true,
                                             display_name='true salient regions')
sr.image_diff(elements_visualized, trueelements_visualized)

False